In [1]:
import numpy as np
import pandas as pd
import ast
from datetime import datetime
import re
import pyarrow as pa
import pyarrow.parquet as pq

In [86]:
archivo= pd.read_csv('../steam_games/extraccion_steam_games.ipynb')

In [87]:
df_games= pd.DataFrame(archivo)

In [88]:
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"('Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"('Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,"('Single-player',)",4.99,0.0,761140.0,Kotoshiro
1,"Making Fun, Inc.","('Free to Play', 'Indie', 'RPG', 'Strategy')",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"('Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"('Single-player', 'Multi-player', 'Online Mult...",Free To Play,0.0,643980.0,Secret Level SRL
2,Poolians.com,"('Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"('Free to Play', 'Simulation', 'Sports', 'Casu...",http://steamcommunity.com/app/670290/reviews/?...,"('Single-player', 'Multi-player', 'Online Mult...",Free to Play,0.0,670290.0,Poolians.com
3,彼岸领域,"('Action', 'Adventure', 'Casual')",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"('Action', 'Adventure', 'Casual')",http://steamcommunity.com/app/767400/reviews/?...,"('Single-player',)",0.99,0.0,767400.0,彼岸领域
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"('Action', 'Indie', 'Casual', 'Sports')",http://steamcommunity.com/app/773570/reviews/?...,"('Single-player', 'Full controller support', '...",2.99,0.0,773570.0,NaN


In [89]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24073 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.2+ MB


In [90]:
# veamos la comparacion de las columnas title y app_name, ya que por denominacion, parece que refieren al mismo dato...
temp= df_games[(df_games['title'] == df_games['app_name']) == False]

In [91]:
# analicemos los registros en que estos dos campos son diferentes, pero en inguno de los dos se presentan valores nan
temp[['title', 'app_name']].dropna()
# vemos que ambos campos refieren al mismo dato, solo con pequeñas diferencias en errores relacionados al caracter '&' en el campo 'title', quizas provocados por el tipo de encoding.


,title,app_name
80,Sam &amp; Max 101: Culture Shock,Sam & Max 101: Culture Shock
83,Sam &amp; Max 102: Situation: Comedy,Sam & Max 102: Situation: Comedy
109,Command &amp; Conquer: Red Alert 3,Command & Conquer: Red Alert 3
182,Heroes of Might &amp; Magic V: Hammers of Fate,Heroes of Might & Magic V: Hammers of Fate
184,Heroes of Might &amp; Magic V: Tribes of the East,Heroes of Might & Magic V: Tribes of the East
...,...,...
31871,Sam &amp; Max 105: Reality 2.0,Sam & Max 105: Reality 2.0
31872,Sam &amp; Max 104: Abe Lincoln Must Die!,Sam & Max 104: Abe Lincoln Must Die!
31873,Sam &amp; Max 106: Bright Side of the Moon,Sam & Max 106: Bright Side of the Moon
31898,Making History: The Calm &amp; the Storm,Making History: The Calm & the Storm


In [92]:
df_games.isna().sum()

publisher       8062
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

In [93]:
# lo que haremos en este caso es rellenar los valores nan del campo 'app_name' qu es el que cuenta con menor cantidad de valores nan
# con los valores que si estan presentes en el campo 'title'
# identifiquemos el indice de los valores nan presentes en el campo 'app_name'
df_games[df_games['app_name'].isna()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
74,NaN,NaN,NaN,NaN,http://store.steampowered.com/,NaN,NaN,NaN,NaN,19.99,0.0,NaN,NaN
2580,NaN,"('Action', 'Indie')",NaN,NaN,http://store.steampowered.com/app/317160/_/,2014-08-26,"('Action', 'Indie')",http://steamcommunity.com/app/317160/reviews/?...,"('Single-player', 'Game demo')",NaN,0.0,317160.0,NaN


In [94]:
# vemos que estos registros no cuentan con ningun valor en los campos 'title' y 'appn_name', por lo que no tenemos referencia del juego al que refieren, por lo que los borraremos.
df_games = df_games.drop(index= [74,2580]).reset_index(drop= True)


In [96]:
#vemos la cantidad de valores nan en la columna 'id'
df_games['id'].isna().sum()

# asignamos para los valores nan el valor -1 que no hace referencia a ningun juego.
df_games['id'] = df_games['id'].fillna(-1)

#transformamos el tipo de dato de la columna 'id' a int
df_games['id'] = df_games['id'].apply(lambda x: int(x))

In [99]:
df_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"('Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"('Strategy', 'Action', 'Indie', 'Casual', 'Sim...",http://steamcommunity.com/app/761140/reviews/?...,"('Single-player',)",4.99,0.0,761140,Kotoshiro
1,"Making Fun, Inc.","('Free to Play', 'Indie', 'RPG', 'Strategy')",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"('Free to Play', 'Strategy', 'Indie', 'RPG', '...",http://steamcommunity.com/app/643980/reviews/?...,"('Single-player', 'Multi-player', 'Online Mult...",Free To Play,0.0,643980,Secret Level SRL


In [100]:
df_games['release_date']= df_games['release_date'].fillna('dato invalido')

In [101]:
a= [(x.startswith('2') != True and x.startswith('1') != True and x.startswith('dato invalido') != True) for x in df_games['release_date']]
df_games[a]


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
10,Qucheza,"('Action', 'Indie', 'Simulation', 'Early Access')",Uncanny Islands,Uncanny Islands,http://store.steampowered.com/app/768570/Uncan...,Soon..,"('Early Access', 'Action', 'Indie', 'Simulatio...",http://steamcommunity.com/app/768570/reviews/?...,"('Single-player',)",NaN,1.0,768570,Qucheza
44,zha7,"('Adventure', 'Free to Play', 'RPG', 'Early Ac...",Idle Adventure,Idle Adventure,http://store.steampowered.com/app/714020/Idle_...,Beta测试已开启,"('Early Access', 'Free to Play', 'Adventure', ...",http://steamcommunity.com/app/714020/reviews/?...,"('MMO', 'Steam Achievements')",NaN,1.0,714020,"zha7,Phaedruns,Yingke Wu"
249,Erik Svedäng,"('Adventure', 'Indie')",Blueberry Garden,Blueberry Garden,http://store.steampowered.com/app/29160/Bluebe...,Jun 2009,"('Adventure', 'Indie', 'Short', 'Puzzle')",http://steamcommunity.com/app/29160/reviews/?b...,"('Single-player',)",4.99,0.0,29160,Erik Svedäng
505,Paradox Interactive,"('RPG', 'Strategy')",Age of Wonders II: The Wizard's Throne,Age of Wonders II: The Wizard's Throne,http://store.steampowered.com/app/61510/Age_of...,Oct 2010,"('Strategy', 'RPG', 'Turn-Based Strategy', 'Fa...",http://steamcommunity.com/app/61510/reviews/?b...,"('Single-player', 'Multi-player')",9.99,0.0,61510,Triumph Studios
508,Paradox Interactive,"('RPG', 'Strategy')",Age of Wonders Shadow Magic,Age of Wonders Shadow Magic,http://store.steampowered.com/app/61520/Age_of...,Oct 2010,"('Strategy', 'RPG', 'Turn-Based Strategy', 'Fa...",http://steamcommunity.com/app/61520/reviews/?b...,"('Single-player', 'Multi-player')",9.99,0.0,61520,Triumph Studios
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32056,Windy Hill,"('Action', 'Adventure', 'Indie', 'Early Access')",Orphan,Orphan,http://store.steampowered.com/app/549700/Orphan/,Winter 2018,"('Early Access', 'Action', 'Adventure', 'Indie')",http://steamcommunity.com/app/549700/reviews/?...,"('Single-player', 'Steam Achievements', 'Full ...",NaN,1.0,549700,Windy Hill
32079,Fat Dog Games,"('Action', 'Adventure', 'Indie', 'RPG')",Sand is the Soul,Sand is the Soul,http://store.steampowered.com/app/716780/Sand_...,Q1 2018,"('Action', 'Adventure', 'RPG', 'Indie')",http://steamcommunity.com/app/716780/reviews/?...,"('Single-player', 'Steam Achievements')",NaN,0.0,716780,MGP Studios
32081,Wrecked Angle Studios,"('Action', 'Casual', 'Indie', 'Early Access')",Chromalocity,Chromalocity,http://store.steampowered.com/app/718250/Chrom...,"Q1, 2018","('Early Access', 'Action', 'Indie', 'Casual', ...",http://steamcommunity.com/app/718250/reviews/?...,"('Single-player', 'Full controller support', '...",NaN,1.0,718250,Wrecked Angle Studios
32085,"Greyson Richey, Nicholas Lives","('Action', 'Casual', 'Indie', 'Simulation')",Lonely Astronaut,Lonely Astronaut,http://store.steampowered.com/app/755830/Lonel...,Coming Soon,"('Casual', 'Action', 'Indie', 'Simulation')",http://steamcommunity.com/app/755830/reviews/?...,"('Single-player',)",NaN,0.0,755830,"Greyson Richey,Nicholas Lives"


In [102]:
df_games['release_date'] = df_games['release_date'].apply(lambda x: x.split('-')[0].replace("'",''))
df_games['release_date'][a] = df_games['release_date'].apply(lambda x: str(x).split())

C:\Users\usuario\AppData\Local\Temp\ipykernel_6180\2152669988.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games['release_date'][a] = df_games['release_date'].apply(lambda x: str(x).split())


In [103]:
for x in df_games['release_date'][a].index:
    try:
        df_games['release_date'][x] = df_games['release_date'][x][1]
    except Exception:
        continue


C:\Users\usuario\AppData\Local\Temp\ipykernel_6180\694787284.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games['release_date'][x] = df_games['release_date'][x][1]


In [104]:
df_games['release_date'][a].value_counts()

2017                 48
2018                 33
Soon                 28
[TBA]                16
2014                 12
soon                 10
2016                  9
2015                  7
[TBD]                 6
2010                  6
[SOON]                5
its                   4
2009                  4
[Soon]                4
it                    2
be                    2
Soon!                 2
Access                2
yet                   2
soon..                1
Date                  1
Q1                    1
Be                    1
Q3                    1
[Soon..]              1
(ish),                1
[Soon™]               1
2013                  1
2012                  1
Fall                  1
late                  1
Early                 1
Spring                1
["""Soon"""]          1
Beta                  1
is                    1
conflicted.           1
2019                  1
bientôt...            1
coming                1
[Beta测试已开启]           1
̴0̶0̶1̶1̶0̷0̶1̵1

In [105]:
for idx, x in enumerate(df_games['release_date']):
    if not str(x).isdigit():
        df_games.loc[idx, 'release_date'] = 'dato invalido'
    else:
        df_games.loc[idx, 'release_date'] = int(x)

In [106]:
df_games['release_date']

0                 2018
1                 2018
2                 2017
3                 2017
4        dato invalido
             ...      
32128             2018
32129             2018
32130             2018
32131             2017
32132    dato invalido
Name: release_date, Length: 32133, dtype: object

In [107]:
df_games.price

0                4.99
1        Free To Play
2        Free to Play
3                0.99
4                2.99
             ...     
32128            1.99
32129            4.99
32130            1.99
32131            4.99
32132            4.99
Name: price, Length: 32133, dtype: object

In [108]:
# for x in df_games:
#     try:
#         if x == np.nan:
#             df_games['price'] = 0.0
#         df_games['price'] = df_games['price'].astype('float64')
#     except Exception:
#         df_games['price'] = 0.0
# df_games['price'].fillna(0.0, inplace=True)
# for idx,x in enumerate(df_games['price']):
#     if type(x) == str:
#         df_games['price'][idx] = 0.0
#     else:
#         continue

def convertidor_float(valor) -> float:
    if pd.isna(valor):
        return 0.0
    try:
        decimal = float(valor)
        return decimal
    except:
        return 0.0

df_games['price']= df_games['price'].apply(lambda x: convertidor_float(x))
df_games['price'] = df_games['price'].astype('float64')

In [109]:
df_games['price']

0        4.99
1        0.00
2        0.00
3        0.99
4        2.99
         ... 
32128    1.99
32129    4.99
32130    1.99
32131    4.99
32132    4.99
Name: price, Length: 32133, dtype: float64

In [110]:
df_games['price'].dtype

dtype('float64')

In [111]:
df_games['price'].isna().sum()

0

In [112]:
df_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [113]:
df_games['publisher'] = df_games['publisher'].fillna('no hay dato')
df_games['developer'] = df_games['developer'].fillna('no hay dato')


In [114]:
# por lo que vemos, algo similar ocurre con los campos 'genres' y 'tags', en donde existe una clara correlacion de los valores de sus datos.

df_games[['genres', 'tags']].dropna()

,genres,tags
0,"('Action', 'Casual', 'Indie', 'Simulation', 'S...","('Strategy', 'Action', 'Indie', 'Casual', 'Sim..."
1,"('Free to Play', 'Indie', 'RPG', 'Strategy')","('Free to Play', 'Strategy', 'Indie', 'RPG', '..."
2,"('Casual', 'Free to Play', 'Indie', 'Simulatio...","('Free to Play', 'Simulation', 'Sports', 'Casu..."
3,"('Action', 'Adventure', 'Casual')","('Action', 'Adventure', 'Casual')"
5,"('Action', 'Adventure', 'Simulation')","('Action', 'Adventure', 'Simulation', 'FPS', '..."
...,...,...
32127,"('Action', 'Adventure', 'Casual', 'Indie')","('Action', 'Indie', 'Casual', 'Violent', 'Adve..."
32128,"('Casual', 'Indie', 'Simulation', 'Strategy')","('Strategy', 'Indie', 'Casual', 'Simulation')"
32129,"('Casual', 'Indie', 'Strategy')","('Strategy', 'Indie', 'Casual')"
32130,"('Indie', 'Racing', 'Simulation')","('Indie', 'Simulation', 'Racing')"


In [115]:
# vemos que existe una correlacion entre los generos y los tags asociados a los juegos, 

In [116]:
df_games[['genres', 'tags']].isna().sum()

genres    3282
tags       162
dtype: int64

In [117]:
# primero reemplazamos los valores nan con el valor 'no hay dato'

df_games['genres'] = df_games['genres'].fillna('no hay dato')
df_games['tags'] = df_games['tags'].fillna('no hay dato')

In [118]:
def safe_eval(x):
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return np.nan  # Otra opción podría ser retornar NaN usando np.nan

# Aplicar la función safe_eval a las columnas 'genres' y 'tags'
df_games[['genres', 'tags']] = df_games[['genres', 'tags']].applymap(safe_eval)

In [119]:
df_games = df_games.explode('genres')
df_games = df_games.dropna(subset=['genres'])
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018,"(Strategy, Action, Indie, Casual, Simulation)",http://steamcommunity.com/app/761140/reviews/?...,"('Single-player',)",4.99,0.0,761140,Kotoshiro
0,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018,"(Strategy, Action, Indie, Casual, Simulation)",http://steamcommunity.com/app/761140/reviews/?...,"('Single-player',)",4.99,0.0,761140,Kotoshiro
0,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018,"(Strategy, Action, Indie, Casual, Simulation)",http://steamcommunity.com/app/761140/reviews/?...,"('Single-player',)",4.99,0.0,761140,Kotoshiro
0,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018,"(Strategy, Action, Indie, Casual, Simulation)",http://steamcommunity.com/app/761140/reviews/?...,"('Single-player',)",4.99,0.0,761140,Kotoshiro
0,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018,"(Strategy, Action, Indie, Casual, Simulation)",http://steamcommunity.com/app/761140/reviews/?...,"('Single-player',)",4.99,0.0,761140,Kotoshiro


In [120]:
df_games = df_games.drop(['tags', 'specs', 'url', 'reviews_url', 'title'], axis=1)

In [122]:
df_games.to_csv('ETL2_steam_games.csv', index= False)

In [ ]:
pq.write_table(pa.Table.from_pandas(df_games), '../data/ETL2_steam_games.parquet')

Aclaraciones:

* Podriamos tratar los datos del campo 'genres' y 'tags' como iguales, ya que las etiquetas de genero en su mayoria, estan incluidas como tags en el campo analogo. Esto nos permitiria obtener los valores analogos del campo 'tags' para el campo 'genres' donde este ultimo no cuenta con valores, lamentablemente el poder de computo y los recursos disponibles con mi laptop, no me permitieron realizar el cambio. Pero ese tratamiento de los datos es posible y el mejor curso de accion para ellos.
